In [ ]:
import pandas as pd
import numpy as np
import duckdb as ddb
# import plotly.express as px
import matplotlib.pyplot as plt
import dotenv

In [2]:
env = dotenv.dotenv_values(".env")

In [3]:
con = ddb.connect()

In [4]:
con.install_extension('httpfs')
con.load_extension('httpfs')

In [5]:
con.sql(f"""
CREATE OR REPLACE SECRET (
    TYPE r2,
    KEY_ID '{env['R2_ACCESS_KEY_ID']}',
    SECRET '{env['R2_SECRET_ACCESS_KEY']}',
    ACCOUNT_ID '{env['R2_ACCOUNT_ID']}'
);
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ true    │
└─────────┘

In [ ]:
BUCKET = "financial-data-store"
ASSET = "mutual_funds"
CATEGORY = "clean"
PATH = f"r2://{BUCKET}/{ASSET}/{CATEGORY}/"

In [7]:
nav_df = con.execute(f"""
    SELECT * 
    FROM read_parquet('{PATH}nav_daily_growth_plan.parquet')
""").df()

: 

In [ ]:
PATH

'r2://mf-data-pipeline/mutual_funds/clean/'

In [29]:
nav_df.head()

,scheme_code,isin_growth,isin_dividend,nav,date,amc_name,scheme_name,scheme_type,scheme_category,scheme_nav_name,scheme_category_level1,scheme_category_level2,is_direct,is_growth_plan
0,149303,INF194KB1CL0,None,17.8140,2025-08-28,Bandhan AMC Limited,BANDHAN MULTI CAP FUND,Open Ended,Equity Scheme - Multi Cap Fund,BANDHAN MULTI CAP FUND - GROWTH - DIRECT PLAN,Equity Scheme,Multi Cap Fund,True,True
1,151812,INF769K01KH4,None,13.9970,2025-08-28,Mirae Asset Investment Managers (India) Pvt. Ltd,Mirae Asset Multicap Fund,Open Ended,Equity Scheme - Multi Cap Fund,Mirae Asset Multicap Fund - Regular Plan - Growth,Equity Scheme,Multi Cap Fund,False,True
2,100033,INF209K01165,None,883.6500,2025-08-28,Aditya Birla Sun Life AMC Limited,Aditya Birla Sun Life Large & Mid Cap Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Aditya Birla Sun Life Large & Mid Cap Fund - R...,Equity Scheme,Large & Mid Cap Fund,False,True
3,118510,INF090I01IN1,None,209.6454,2025-08-28,Franklin Templeton Asset Management (India) Pr...,Franklin India Large & Mid Cap Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Franklin India Large & Mid Cap Fund - Direct -...,Equity Scheme,Large & Mid Cap Fund,True,True
4,152941,INF0R8701269,None,10.6400,2025-08-28,Helios Capital Asset Management (India) Pvt. Ltd.,Helios Large & Mid Cap Fund,Open Ended,Equity Scheme - Large & Mid Cap Fund,Helios Large & Mid Cap Fund - Direct Plan - Gr...,Equity Scheme,Large & Mid Cap Fund,True,True


In [ ]:
analysis_df = (nav_df
               .query("(scheme_category_level1 == 'Equity Scheme') & (is_direct == True)")
               .sort_values(['scheme_code', 'date'], ascending=[True, True])
               .assign(perc_return=lambda x: x.groupby('scheme_code')['nav'].pct_change(),
                       log_return=lambda x: np.log1p(x['perc_return'])
                       )
               )

/home/modiis/projects/mf_data_pipeline/.venv/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [ ]:
group_cols = ['scheme_code', 'isin_growth', 'isin_dividend',
              'amc_name', 'scheme_name', 'scheme_type', 'scheme_category',
              'scheme_nav_name', 'scheme_category_level1', 'scheme_category_level2',
              'is_direct', 'is_growth_plan']

In [ ]:
annual_df = (analysis_df
             .groupby([*group_cols, pd.Grouper(key='date', freq='YE')], dropna=False)['log_return'].sum()
             .reset_index()
             )

In [ ]:
annual_df.to_clipboard()

: 

In [54]:
analysis_df.columns

Index(['scheme_code', 'isin_growth', 'isin_dividend', 'nav', 'date',
       'amc_name', 'scheme_name', 'scheme_type', 'scheme_category',
       'scheme_nav_name', 'scheme_category_level1', 'scheme_category_level2',
       'is_direct', 'is_growth_plan', 'perc_return', 'log_return'],
      dtype='object')

In [55]:
analysis_df

,scheme_code,isin_growth,isin_dividend,nav,date,amc_name,scheme_name,scheme_type,scheme_category,scheme_nav_name,scheme_category_level1,scheme_category_level2,is_direct,is_growth_plan,perc_return,log_return
3503731,103490,INF082J01036,None,10.29,2006-04-03,Quantum Asset Management Company Private Limited,Quantum Value Fund,Open Ended,Equity Scheme - Value Fund,Quantum Value Fund - Direct Plan Growth Option,Equity Scheme,Value Fund,True,True,NaN,NaN
3503732,103490,INF082J01036,None,10.35,2006-04-04,Quantum Asset Management Company Private Limited,Quantum Value Fund,Open Ended,Equity Scheme - Value Fund,Quantum Value Fund - Direct Plan Growth Option,Equity Scheme,Value Fund,True,True,0.005831,0.005814
6234898,103490,INF082J01036,None,10.37,2006-04-05,Quantum Asset Management Company Private Limited,Quantum Value Fund,Open Ended,Equity Scheme - Value Fund,Quantum Value Fund - Direct Plan Growth Option,Equity Scheme,Value Fund,True,True,0.001932,0.001931
2246590,103490,INF082J01036,None,10.33,2006-04-07,Quantum Asset Management Company Private Limited,Quantum Value Fund,Open Ended,Equity Scheme - Value Fund,Quantum Value Fund - Direct Plan Growth Option,Equity Scheme,Value Fund,True,True,-0.003857,-0.003865
5487284,103490,INF082J01036,None,10.38,2006-04-10,Quantum Asset Management Company Private Limited,Quantum Value Fund,Open Ended,Equity Scheme - Value Fund,Quantum Value Fund - Direct Plan Growth Option,Equity Scheme,Value Fund,True,True,0.004840,0.004829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4144142,153900,INF205KA1BI6,None,10.02,2025-10-29,Invesco Asset Management (India) Private Limited,Invesco India Consumption Fund,Open Ended,Equity Scheme - Sectoral/ Thematic,Invesco India Consumption Fund - Direct Plan -...,Equity Scheme,Sectoral/ Thematic,True,True,NaN,NaN
3602286,153900,INF205KA1BI6,None,10.02,2025-10-30,Invesco Asset Management (India) Private Limited,Invesco India Consumption Fund,Open Ended,Equity Scheme - Sectoral/ Thematic,Invesco India Consumption Fund - Direct Plan -...,Equity Scheme,Sectoral/ Thematic,True,True,0.000000,0.000000
6073282,153900,INF205KA1BI6,None,9.95,2025-10-31,Invesco Asset Management (India) Private Limited,Invesco India Consumption Fund,Open Ended,Equity Scheme - Sectoral/ Thematic,Invesco India Consumption Fund - Direct Plan -...,Equity Scheme,Sectoral/ Thematic,True,True,-0.006986,-0.007011
2218578,153900,INF205KA1BI6,None,9.96,2025-11-03,Invesco Asset Management (India) Private Limited,Invesco India Consumption Fund,Open Ended,Equity Scheme - Sectoral/ Thematic,Invesco India Consumption Fund - Direct Plan -...,Equity Scheme,Sectoral/ Thematic,True,True,0.001005,0.001005
